In [1]:
# dependencies
import tensorflow_datasets as tfds
import tensorflow as tf

# mount google drive when running notebook in google colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# load civil_comments dataset, write train and test data to two separate tf.data objects
dataset, info = tfds.load('civil_comments', with_info=True,
                          as_supervised=True)


/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/civil_comments/0.9.0.incompleteTQ2L8W/civil_comments-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/civil_comments/0.9.0.incompleteTQ2L8W/civil_comments-validation.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/civil_comments/0.9.0.incompleteTQ2L8W/civil_comments-test.tfrecord


Dataset civil_comments downloaded and prepared to /root/tensorflow_datasets/civil_comments/0.9.0. Subsequent calls will reuse this data.


In [0]:
train_examples, test_examples = dataset['train'], dataset['test']

In [0]:
# see first 10 entries in train_examples
i = 0
for comment, label in train_examples:
    print(tf_encode(comment, label))

    i += 1
    if i == 10:
        break

(<tf.Tensor: shape=(67,), dtype=int64, numpy=
array([4116, 3958, 2952,    2, 2946,    2,   50,  258,  803,  882,  288,
          9,  260,  144, 2757, 3870, 3870,   14, 3311,  928, 3892,  893,
        125,  106,   14,  315, 3929, 3278, 3153, 2249, 3892,   48,    1,
        171,    3,  235, 2584,   29, 1574,  194, 3933, 3899,   18,  753,
       1739,    1,  552, 1936, 3892,   83, 1089,   30,   85, 1846,   10,
          1, 3342, 3892,   58,   72, 1109,  860, 1241,  277, 3975, 2839,
       4117])>, <tf.Tensor: shape=(1,), dtype=int64, numpy=array([0])>)
(<tf.Tensor: shape=(68,), dtype=int64, numpy=
array([4116,   97,   42,  324,   17,   71, 3899,   15, 2117,  200,   62,
        118,  799,  598,  177,  663,  264,  475, 2839, 3275,  811, 2803,
       2839,  241, 3907, 1934, 3907,  491, 1393,  187, 3905, 3957, 3469,
       3522, 3907, 2149, 3905, 3915, 3916, 3915, 3905, 1735, 3905, 3212,
       3905,  859, 3905,  295, 3905,  244,  335, 3905, 1244, 3905, 3009,
        785,  470, 3905, 1114, 39

In [0]:
# create tokenizer to encode comments. use tokenizer.decode() to translate from tokenized string. vocab size around 4000 for computational speed
tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    (comment.numpy() for comment, toxicity in train_examples), target_vocab_size=2**12)

In [0]:
import os
#save tokenizer for later
tokenizer_path = os.path.join('content', 'drive', 'My Drive', 'civil_comments_small_tokenizer')
tokenizer.save_to_file(tokenizer_path)

In [0]:
# load tokenizer settings file
import os
tokenizer_path = '/content/drive/My Drive/ML_settings/civil_comments_small_tokenizer'
tokenizer = tfds.features.text.SubwordTextEncoder.load_from_file('/content/drive/My Drive/ML_settings/civil_comments_small_tokenizer')

In [0]:
# set some hyperparameters
BUFFER_SIZE = 10000
BATCH_SIZE = 300
MAX_LENGTH = 40

#function to add start and end tokens to entries. returns list of tokens
def encode(comment, label):
  # adds token to the start and ends of each comment
  comment = [tokenizer.vocab_size] + tokenizer.encode(comment.numpy()) + [tokenizer.vocab_size+1]

  return comment, [label] 

# function to encode each comment 
def tf_encode(comment, label):
  result_comment, result_label = tf.py_function(encode, [comment, label], [tf.int64, tf.int64])
  result_comment.set_shape([None])
  result_label.set_shape([None])
  return result_comment, result_label

# function to drop comments over a certain length for computational purposes
def filter_max_length(x, y, max_length=MAX_LENGTH):
  return tf.logical_and(tf.size(x) <= max_length,
                        tf.size(y) <= max_length)

# function to turn label probabilities into binary labels
def binary_label(comment, label):
  if label >= 0.5:
    label = 1.
    return comment, label
  label = 0.
  return comment, label

In [0]:
# apply encoding and filtering to train and test datasets

train_preprocessed = (train_examples
    .map(binary_label)
    .map(tf_encode)
    .filter(filter_max_length)
    # cache the dataset to memory to get a speedup while reading from it.
    .cache()
    .shuffle(BUFFER_SIZE))

test_preprocessed = (
    test_examples
    .map(binary_label)
    .map(tf_encode)
    .filter(filter_max_length)
    .cache()
    .shuffle(BUFFER_SIZE))      

In [0]:
# pad each comment with 0s to the same batch size

train_dataset = (train_preprocessed
                 .shuffle(BUFFER_SIZE)
                 .padded_batch(BATCH_SIZE, padded_shapes=([None],[None])))

test_dataset = (test_preprocessed
                .padded_batch(BATCH_SIZE,  padded_shapes=([None],[None])))

In [0]:
"""
Create model layers:
Embedding layer has 64 dimensions per word in vocabulary
Bidirectional layers for LSTM
"""

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size+2, 64), 
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(8)), # LSTM layer for RNN
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [0]:
# compile model
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [0]:
history = model.fit(train_dataset, epochs=1,
                    validation_data=test_dataset, 
                    validation_steps=30)

2095/2095 [==============================] - 1627s 777ms/step - loss: 0.1788 - accuracy: 0.9391 - val_loss: 0.1467 - val_accuracy: 0.9479


In [0]:
#save the model
import os
os.path.join('content', 'drive', 'My Drive')
model.save('/content/drive/My Drive/ML_settings/small_rnn.h5') 

In [0]:
# load model from .h5

model = tf.keras.models.load_model('/content/drive/My Drive/ML_settings/small_rnn.h5')

In [0]:
# function for padding out an input string with 0s
def pad_to_size(vec, size):
  zeros = [0] * (size - len(vec))
  vec.extend(zeros)
  return vec

# create a function for predicting toxicity from a string. Add token padding by default
def sample_predict(sample_pred_text, pad=True):
  encoded_sample_pred_text = tokenizer.encode(sample_pred_text)

  if pad:
    encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text, 64) # call pad_to_size to be at least 64 dimensions long
  encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
  predictions = model.predict(tf.expand_dims(encoded_sample_pred_text, 0))

  return (predictions)

In [0]:
# test accuracy of model
results = model.evaluate(test_dataset, verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

114/114 - 57s - loss: 0.1473 - accuracy: 0.9463
loss: 0.147
accuracy: 0.946


In [14]:
# quick test of model prediction
sample_predict('you are a kind person')

array([[-0.04236663]], dtype=float32)

In [24]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

#write out tensors and metadata for use in tensorboard embedding projector 
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for num, word in enumerate(tokenizer.subwords):
  vec = weights[num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()

(4118, 64)
